In [22]:
#!/usr/bin/env python3
#pip install  rioxarray==0.3.1
import pandas as pd
import xarray as xr
import os
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
import matplotlib.colors
scriptsdir = os.getcwd()
from scipy.interpolate import griddata
from functools import reduce
import itertools
import argparse
import pickle


scenario = ["rcp26"]
time = [65]


sdms = ["GAM", "GBM"]
gcms = ['GFDL-ESM2M', 'IPSL-CM5A-LR', 'HadGEM2-ES', 'MIROC5']
taxas = ["Mammals","Amphibians","Bird"]  # Add the second taxa here
#scenario = "rcp26"
#time=35
sdms = ["GAM", "GBM"]
gcms = ['GFDL-ESM2M', 'IPSL-CM5A-LR', 'HadGEM2-ES', 'MIROC5']
taxas = ["Mammals","Amphibians","Bird"]  # Add the second taxa here
#scenario = "rcp26"
#time=35
historical_time=1146

years = ['1845', '1990', '1995', '2009', '2010', '2020', '2026', '2032', '2048', '2050',
         '2052', '2056', '2080', '2100', '2150', '2200', '2250']

def detect_outliers_iqr(data_array, multiplier=3):
    """Detect outliers based on the interquartile range (IQR).
       Ignores NaN values and returns information about non-NaN outliers."""
    # Flatten the data array and remove NaN values
    flattened_data = data_array.values.flatten()
    non_nan_data = flattened_data[np.isfinite(flattened_data)]

    # Calculate IQR
    q1 = np.percentile(non_nan_data, 25)
    q3 = np.percentile(non_nan_data, 75)
    iqr = q3 - q1
    lower_bound = q1 - multiplier * iqr
    upper_bound = q3 + multiplier * iqr

    # Detect outliers
    outlier_condition = (data_array < lower_bound) | (data_array > upper_bound)
    outliers = data_array.where(outlier_condition, drop=True)

    if outliers.size > 0:
        outlier_info = {
            'count': outliers.size,
            'example_values': outliers.values.flatten()[:5],
            'coordinates': [{'lat': lat, 'lon': lon} for lat, lon in zip(outliers.lat.values, outliers.lon.values)]
        }
        return True, outlier_info
    return False, None


# Create the dictionaries
newvalue_hist_sum = {}
newvalue_future_sum = {}
sumbin_hist_sum = {}
sumbin_future_sum = {}

# Initialize the dictionaries with SDM, GCM, and taxa keys
mean_newvalue_change = {}
mean_sum_bin_change = {}
mean_land_use_change = {}

for sdm in sdms:
    newvalue_hist_sum[sdm] = {}
    newvalue_future_sum[sdm] = {}
    sumbin_hist_sum[sdm] = {}
    sumbin_future_sum[sdm] = {}
    mean_newvalue_change[sdm] = {}
    mean_sum_bin_change[sdm] = {}
    mean_land_use_change[sdm] = {}

    for gcm in gcms:
        newvalue_hist_sum[sdm][gcm] = {}
        newvalue_future_sum[sdm][gcm] = {}
        sumbin_hist_sum[sdm][gcm] = {}
        sumbin_future_sum[sdm][gcm] = {}
        mean_newvalue_change[sdm][gcm] = {}
        mean_sum_bin_change[sdm][gcm] = {}
        mean_land_use_change[sdm][gcm] = {}

        for taxa in taxas:
            newvalue_hist_sum[sdm][gcm][taxa] = []
            newvalue_future_sum[sdm][gcm][taxa] = []
            sumbin_hist_sum[sdm][gcm][taxa] = []
            sumbin_future_sum[sdm][gcm][taxa] = []

# Loop over all taxa
for taxa in taxas:
    for sdm in sdms:
        dir_species = "/storage/scratch/users/ch21o450/data/LandClim_Output/" + sdm + "/" + taxa + "/EWEMBI/"
        available_file = os.listdir(dir_species)
        available_names = [x.split("_[1146].nc")[0] for x in available_file]

        species_names = available_names[:100]
        # Define the netCDF file path
        netcdf_path_format_future = "/storage/scratch/users/ch21o450/data/LandClim_Output/{}/{}/{}/{}/{}_[{}].nc"
        netcdf_path_format_hist = "/storage/scratch/users/ch21o450/data/LandClim_Output/{}/{}/EWEMBI/{}_[{}].nc"

        # Loop over all species
        for species_name in species_names:
            # Loop over all models
            for gcm in gcms:
                try:
                    # Open the netCDF files
                    ds_newvalue_fut = xr.open_dataset(
                        netcdf_path_format_future.format(sdm, taxa, gcm, scenario[0], species_name, time[0]),
                        decode_times=False
                    )
                    ds_newvalue_hist = xr.open_dataset(
                        netcdf_path_format_hist.format(sdm, taxa, species_name, historical_time),
                        decode_times=False
                    )
                    
                    # Get the newvalue and sum_bin
                    newvalue_hist = ds_newvalue_hist["newvalue"]
                    newvalue_future = ds_newvalue_fut["newvalue"]
                    sum_bin_hist = ds_newvalue_hist["sum_bin"].isel(time=0)
                    sum_bin_future = ds_newvalue_fut["sum_bin"].isel(time=0)

                    # Append the newvalue to the dictionaries
                    newvalue_hist_sum[sdm][gcm][taxa].append(newvalue_hist)
                    newvalue_future_sum[sdm][gcm][taxa].append(newvalue_future)
                    sumbin_hist_sum[sdm][gcm][taxa].append(sum_bin_hist)
                    sumbin_future_sum[sdm][gcm][taxa].append(sum_bin_future)

                    # Your existing code to process the files goes here

                except FileNotFoundError:
                    # Handle the case where the file is not found
                    print(f"File not found for species {species_name}. Continuing to the next species.")
                    continue
                except Exception as e:
                    # Handle other exceptions if needed
                    print(f"An error occurred for species {species_name}: {e}")
                    continue



# Output directory for pickles

#output_dir = "/storage/scratch/users/ch21o450/data/intermediate_results/"
output_dir = "/storage/scratch/users/ch21o450/data/intermediate_results/"
os.makedirs(output_dir, exist_ok=True)

# Loop over all taxa
for taxa in taxas:
    for sdm in sdms:
        for gcm in gcms:
            # Concatenate and sum values
            newvalue_hist_sum_taxa = xr.concat(newvalue_hist_sum[sdm][gcm][taxa], dim="species").sum(dim="species")
            newvalue_future_sum_taxa = xr.concat(newvalue_future_sum[sdm][gcm][taxa], dim="species").sum(dim="species")
            sum_bin_hist_sum_taxa = xr.concat(sumbin_hist_sum[sdm][gcm][taxa], dim="species").sum(dim="species")
            sum_bin_future_sum_taxa = xr.concat(sumbin_future_sum[sdm][gcm][taxa], dim="species").sum(dim="species")

            # Detect outliers in the aggregated data

            is_outlier, outlier_info = detect_outliers_iqr(newvalue_future_sum_taxa)
            if is_outlier:
                print("Outliers Detected:")
                print(f"Count: {outlier_info['count']}")
                print(f"Example Values: {outlier_info['example_values']}")
                print(f"Coordinates: {outlier_info['coordinates']}")  # print first 5 coordinates


Outliers Detected:
Count: 200200
Example Values: [nan nan nan nan nan]
Coordinates: [{'lat': 83.75, 'lon': -179.75}, {'lat': 83.25, 'lon': -179.25}, {'lat': 82.75, 'lon': -178.75}, {'lat': 82.25, 'lon': -178.25}, {'lat': 81.75, 'lon': -177.75}, {'lat': 81.25, 'lon': -177.25}, {'lat': 80.75, 'lon': -176.75}, {'lat': 80.25, 'lon': -176.25}, {'lat': 79.75, 'lon': -175.75}, {'lat': 79.25, 'lon': -175.25}, {'lat': 78.75, 'lon': -174.75}, {'lat': 78.25, 'lon': -174.25}, {'lat': 77.75, 'lon': -173.75}, {'lat': 77.25, 'lon': -173.25}, {'lat': 76.75, 'lon': -172.75}, {'lat': 76.25, 'lon': -172.25}, {'lat': 75.75, 'lon': -171.75}, {'lat': 75.25, 'lon': -171.25}, {'lat': 74.75, 'lon': -170.75}, {'lat': 74.25, 'lon': -170.25}, {'lat': 73.75, 'lon': -167.25}, {'lat': 73.25, 'lon': -166.75}, {'lat': 72.75, 'lon': -166.25}, {'lat': 72.25, 'lon': -165.75}, {'lat': 71.75, 'lon': -165.25}, {'lat': 71.25, 'lon': -164.75}, {'lat': 70.75, 'lon': -164.25}, {'lat': 70.25, 'lon': -163.75}, {'lat': 69.75, 'lon